In [1]:
import agate

In [2]:
drones = agate.Table.from_csv('drones.csv')

tester = agate.TypeTester(force={
    'Id': agate.Text(),
    'ZCTA': agate.Text()
})

zips = agate.Table.from_csv('zcta.csv', column_types=tester)

In [3]:
print(drones)

|----------------------------+---------------|
|  column_names              | column_types  |
|----------------------------+---------------|
|  Country                   | Text          |
|  State / Province / Region | Text          |
|  City                      | Text          |
|  Postal Code               | Text          |
|  Number of Registrations   | Number        |
|----------------------------+---------------|



In [4]:
print(zips)

|---------------+---------------|
|  column_names | column_types  |
|---------------+---------------|
|  Id           | Text          |
|  ZCTA         | Text          |
|  Geography    | Text          |
|  Total        | Number        |
|---------------+---------------|



In [5]:
us_drones = drones.where(lambda row: row['Country'] == 'United States')

In [6]:
drone_with_populations = us_drones.join(zips, 'Postal Code', 'ZCTA', inner=True)

In [7]:
print(drone_with_populations)

|----------------------------+---------------|
|  column_names              | column_types  |
|----------------------------+---------------|
|  Country                   | Text          |
|  State / Province / Region | Text          |
|  City                      | Text          |
|  Postal Code               | Text          |
|  Number of Registrations   | Number        |
|  Id                        | Text          |
|  Geography                 | Text          |
|  Total                     | Number        |
|----------------------------+---------------|



In [8]:
drone_with_populations.print_table(max_rows=10)

|----------------+---------------------------+--------------+-------------+-------------------------+----------------+-------------+---------|
|  Country       | State / Province / Region | City         | Postal Code | Number of Registrations | Id             | Geography   |  Total  |
|----------------+---------------------------+--------------+-------------+-------------------------+----------------+-------------+---------|
|  United States | AK                        | 99508        | 99508       |                       1 | 8600000US99508 | ZCTA5 99508 | 35,135  |
|  United States | AK                        | Aachen       | 52064       |                       1 | 8600000US52064 | ZCTA5 52064 |    885  |
|  United States | AK                        | Alakanuk     | 99554       |                       1 | 8600000US99554 | ZCTA5 99554 |    733  |
|  United States | AK                        | Alaska       | 99712       |                       1 | 8600000US99712 | ZCTA5 99712 | 12,134  |

In [9]:
from decimal import Decimal

def drone_rate(row):
    if row['Number of Registrations'] == 0:
        rate = float(0)
    elif row['Total'] == 0:
        rate = float(0)
    else:
        rate = (row['Number of Registrations']/row['Total'])*100000
    return rate

drone_rate = drone_with_populations.compute([
    ('drone_rate', agate.Formula(agate.Number(), drone_rate))
])

In [10]:
sorted_rates = drone_rate.order_by('drone_rate', reverse=True)

In [14]:
for_printing = sorted_rates.select(['City', 'State / Province / Region', 'Postal Code', 'Number of Registrations','Total', 'drone_rate'])

In [15]:
for_printing.print_table(max_rows=50)

|-----------------------+---------------------------+-------------+-------------------------+-------+---------------------------------|
|  City                 | State / Province / Region | Postal Code | Number of Registrations | Total |                     drone_rate  |
|-----------------------+---------------------------+-------------+-------------------------+-------+---------------------------------|
|  Oakville             | NY                        | 10110       |                       1 |     1 | 100,000.0000000000000000000000  |
|  Pullman              | WA                        | 99164       |                       1 |     1 | 100,000.0000000000000000000000  |
|  Greenville           | MS                        | 38704       |                       1 |     3 |  33,333.3333333333333333333333  |
|  Bellville            | TX                        | 77481       |                       1 |     3 |  33,333.3333333333333333333333  |
|  Thompsons            | TX                    

In [16]:
for_printing.to_csv('droneregistrationrate.csv')